In [1]:
from dataclasses import astuple

import numpy as np
import plotly.graph_objects as go

from src.loaders import FrameLoader, LabelsLoader

In [2]:
frame_loader = FrameLoader('data/a9_r0_dataset_undistorted/images')
labels_loader = LabelsLoader('data/a9_r0_dataset_undistorted/labels')

In [3]:
# Number of labeled vehicles per frame.
label_counts = []
custom_data = []
for labels, labels_data in labels_loader.load_items():
    label_counts.append(len(labels))
    custom_data.append(labels_data['image_file_name'][21:-4])
go.Figure(
    go.Bar(
        y=label_counts,
        customdata=custom_data,
        hovertemplate='index: %{x}<br>#vehicles: %{y}<br>frame: %{customdata}',
    ),
    layout=dict(xaxis_title='frame index', yaxis_title='# labeled vehicles'),
)


In [4]:
# Calculate dimensions stats and count per category.

categories = {}
for labels, _ in labels_loader.load_items():
    for label in labels:
        if label.category not in categories:
            categories[label.category] = []
        categories[label.category].append(label.dimensions)

max_characters = max(len(key) for key in categories.keys())

for category, dimensions in categories.items():
    dimensions = np.array(dimensions)
    means = dimensions.mean(0)
    ranges = np.percentile(dimensions, [1, 99], axis=0).T.tolist()
    print(f'{category} ({dimensions.shape[0]})')
    for i, s in enumerate(('length', 'width ', 'height')):
        print(f'  {s}: Ø {means[i]:.2f} ({ranges[i][0]:.2f} ~> {ranges[i][1]:.2f})')
    print()

CAR (8158)
  length: Ø 4.60 (2.95 ~> 5.53)
  width : Ø 1.94 (1.53 ~> 2.21)
  height: Ø 1.46 (1.19 ~> 1.85)

VAN (1174)
  length: Ø 5.74 (3.80 ~> 8.36)
  width : Ø 2.06 (1.64 ~> 2.59)
  height: Ø 2.17 (1.51 ~> 3.08)

TRAILER (1048)
  length: Ø 11.81 (3.04 ~> 19.40)
  width : Ø 2.52 (1.67 ~> 3.03)
  height: Ø 3.59 (1.35 ~> 4.28)

TRUCK (853)
  length: Ø 5.81 (1.97 ~> 16.11)
  width : Ø 2.51 (1.98 ~> 2.84)
  height: Ø 3.56 (2.00 ~> 4.08)

SPECIAL_VEHICLE (9)
  length: Ø 7.28 (3.46 ~> 10.28)
  width : Ø 2.36 (2.00 ~> 2.75)
  height: Ø 2.87 (1.52 ~> 3.63)

BUS (59)
  length: Ø 13.58 (8.16 ~> 18.77)
  width : Ø 2.61 (2.42 ~> 2.94)
  height: Ø 3.35 (2.97 ~> 3.96)

BICYCLE (1)
  length: Ø 1.42 (1.42 ~> 1.42)
  width : Ø 0.48 (0.48 ~> 0.48)
  height: Ø 1.24 (1.24 ~> 1.24)

MOTORCYCLE (33)
  length: Ø 2.32 (1.65 ~> 4.79)
  width : Ø 0.75 (0.50 ~> 1.76)
  height: Ø 1.64 (1.50 ~> 1.91)

PEDESTRIAN (20)
  length: Ø 0.56 (0.49 ~> 1.52)
  width : Ø 0.50 (0.50 ~> 0.50)
  height: Ø 1.75 (1.73 ~> 1.79)
